In [4]:
import requests
from bs4 import BeautifulSoup
import sqlite3

In [5]:
class Scraping:
    def __init__(self, db_path) -> None:
        self.db_path = db_path
    
    def get_soup(self):
        res = requests.get(f"https://www.data.jma.go.jp/obd/stats/etrn/view/hourly_a1.php?prec_no=43&block_no=0359&year=2024&month=01&day={self.date}&view=")
        soup = BeautifulSoup(res.text, 'html.parser')
        return soup
    
    def get_temp_list(self):
        soup = self.get_soup()
        raw_data = soup.find_all("td", class_="data_0_0")
        temp_list = []
        for i in range(1,240,10):
            temp = raw_data[i]
            temp = temp.text
            temp_list.append(temp)
        return temp_list
    
    def get_hour_list(self):
        hour_list = []
        for i in range(24):
            hour_list.append(i)
        return hour_list

    def comb_data(self, temp:int, hour:int):
        data_list = [self.date, hour, temp]
        return data_list
    
    def insert_data(self, data :list):
        conn = sqlite3.connect(self.db_path)
        cur = conn.cursor()
        cur.execute('INSERT INTO temp_table (date, hour, temperature) VALUES (?,?,?)', (data))
        conn.commit()
        conn.close()

    def create_table(self):
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()

        # テーブルが存在しない場合は作成
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS temp_table (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                date INTEGER,
                hour INTEGER,
                temperature INTEGER
            )
        ''')
        conn.commit()
        conn.close()
    
    def all_insert(self, date:int):
        self.create_table()
        self.date = date
        temp_list = self.get_temp_list()
        hour_list = self.get_hour_list()
        for i in range(len(temp_list)):
            data_4_insert = self.comb_data(temp_list[i], hour_list[i])
            self.insert_data(data_4_insert)

In [8]:
handle = Scraping("/Users/naoki/G2-DSprogramming/G2-DSprogramming/test1.db")
handle.all_insert(15)